## Load Parquet Files:

In [5]:
import pandas as pd
import requests
from io import BytesIO

def load_parquet_from_url(url):
    """
    Downloads a Parquet file from a URL and loads it into a DataFrame.

    Parameters:
    - url (str): The URL to the Parquet file.

    Returns:
    - pd.DataFrame: The loaded DataFrame if successful, None otherwise.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError if status is 4xx, 5xx
        file_stream = BytesIO(response.content)
        customer_reviews_df = pd.read_parquet(file_stream)
        print("File loaded successfully!")
        # display(customer_reviews_df.head())
        return customer_reviews_df
    except requests.exceptions.RequestException as e:
        print(f"Failed to download the file: {e}")
        return None
    except Exception as e:
        print(f"Failed to load the parquet file: {e}")
        return None

## Part1: Verify data counts

In [6]:
import pandas as pd

# List of your parquet files
urls = [
    "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_users.parquet",
    "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_businesses.parquet",
    "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_reviews.parquet"
]

files = [load_parquet_from_url(url) for url in urls]

# Loop through each file and print the record count
for file_path in files:
    try:
        df = pd.read_parquet(file_path)
        record_count = len(df)

        print(f"Successfully loaded '{file_path}'")
        print(f"Number of records: {record_count}\n")

    except Exception as e:
        print(f"Error loading '{file_path}': {e}\n")

File loaded successfully!
File loaded successfully!
File loaded successfully!
Error loading '                      user_id  \
0      qVc8ODYU5SZjKXVBgXdI7w   
1      j14WgRoU_-2ZE1aw1dXrJg   
2      NIhcRW6DWvk1JQhDhXwgOQ   
3      QJI9OSEn6ujRCtrX06vs1w   
4      bNJmIcxc_FRWxftI_vhpbg   
...                       ...   
29781  UFvjDjfbIultYkROA72uyA   
29782  Q8Mtnibb1un87CuBgVM9uA   
29783  1_EPcTpxwg95tg2JdiKuPQ   
29784  DXvHoMQiRRhZqO3ifvJggQ   
29785  T4E07-wgm9L8W8Qaw3AdRA   

                                                 friends  
0      NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...  
1      ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...  
2      T1upaPMzuW7pNj74fO1rjA, CP28puvAEimt4ziuGTDaHA...  
3      RyPeT_ICAtX8ah9dhDpEFw, W8r4aKPZFT3GPIQQDbqB6Q...  
4         -XaJjvMGQNARPz6zDEviaA, idxYND4JfLwijtONahHY6w  
...                                                  ...  
29781                                               None  
29782                                 

## Merge Datasets

In [ ]:
import pandas as pd

# ---- URLs for the full datasets ----
philly_users_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/philly_users.parquet"
philly_reviews_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/philly_reviews.parquet"
philly_businesses_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/philly_businesses.parquet"

# ---- URLs for the filtered datasets ----
filtered_users_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_users.parquet"
filtered_reviews_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_reviews.parquet"
filtered_businesses_url = "https://github.com/tlockhart/project-3/releases/download/v2.0/filtered_restaurants_businesses.parquet"

# ---- Load full datasets ----
philly_users = load_parquet_from_url(philly_users_url)
philly_reviews = load_parquet_from_url(philly_reviews_url)
philly_businesses = load_parquet_from_url(philly_businesses_url)

# ---- Load filtered datasets and select specific columns ----
filtered_users = load_parquet_from_url(filtered_users_url)[["user_id"]]
filtered_reviews = load_parquet_from_url(filtered_reviews_url)[["review_id"]]
filtered_businesses = load_parquet_from_url(filtered_businesses_url)[["business_id"]]

# ---- Merge based on IDs ONLY (inner join) ----
master_users = philly_users.merge(filtered_users, on="user_id", how="inner").reset_index(drop=True)
master_reviews = philly_reviews.merge(filtered_reviews, on="review_id", how="inner").reset_index(drop=True)
master_businesses = philly_businesses.merge(filtered_businesses, on="business_id", how="inner").reset_index(drop=True)

# ---- Saved to github release: Save to new master files ----
# master_users.to_parquet("./Resources/master_users.parquet", index=False)
# master_reviews.to_parquet("./Resources/master_reviews.parquet", index=False)
# master_businesses.to_parquet("./Resources/master_businesses.parquet", index=False)

print("Merged and saved:")
print(f"master_users.parquet: {len(master_users)} records")
print(f"master_reviews.parquet: {len(master_reviews)} records")
print(f"master_businesses.parquet: {len(master_businesses)} records")

File loaded successfully!
File loaded successfully!
File loaded successfully!
File loaded successfully!
File loaded successfully!
File loaded successfully!
Merged and saved:
master_users.parquet: 29786 records
master_reviews.parquet: 85532 records
master_businesses.parquet: 770 records


## Part 2: Merge Data to philly_reviews_full.parquet

In [8]:
import pandas as pd

# Load and reset index properly to prevent index columns from being added to the merge
reviews_df = master_reviews.copy()
users_df = master_users.copy()
businesses_df = master_businesses.copy()


# ✅ Rename columns early
reviews_df = reviews_df.rename(columns={
    "useful": "review_useful",     # Rename useful -> review_useful in reviews
    "stars": "review_stars"
})

businesses_df = businesses_df.rename(columns={
    "name": "business_name"
})

# Optional: check the updated column names
print("Renamed columns:")
print("Reviews Columns:", reviews_df.columns)
print("Users Columns:", users_df.columns)
print("Businesses Columns:", businesses_df.columns)

# Print the first 5 rows of each
print("\nReviews DataFrame (first 5 rows):")
print(reviews_df.head(), "\n")

print("Users DataFrame (first 5 rows):")
print(users_df.head(), "\n")

print("Businesses DataFrame (first 5 rows):")
print(businesses_df.head(), "\n")

# Merge reviews with users on user_id
reviews_users_df = reviews_df.merge(users_df, on="user_id", how="inner")

# Merge the result with businesses on business_id
merged_df = reviews_users_df.merge(businesses_df, on="business_id", how="inner")

# ✅ Filter for reviews with 4.0 stars or more
merged_df = merged_df[merged_df["review_stars"] >= 4.0]

# ✅ Select only the fields you care about
final_df = merged_df[[
    "review_id",
    "user_id",
    "business_id",
    "text",                        # Review text
    "business_name",               # Business name
    "address",                     # Business address
    "city",                        # Business city
    "categories",                  # Business category
    "friends",                     # User friends
    "review_stars",                # Review stars
    "review_useful",               # From reviews table (renamed useful)
    "fans",                        # From users table
    "average_stars",
    "compliment_hot",
    "compliment_more",
    "compliment_profile",
    "compliment_cute",
    "compliment_note",
    "compliment_plain",
    "compliment_cool",
    "compliment_funny",
    "compliment_writer",
    "compliment_photos"
]]

# Safely update friends into a list
final_df.loc[:, "friends"] = final_df["friends"].apply(lambda x: x.split(", ") if isinstance(x, str) and x.strip() else [])

final_df.loc[:, "categories"] = final_df["categories"].apply(
    lambda x: x.split(", ") if isinstance(x, str) and x.strip() else []
)

# Saved to Github Release: Save the cleaned dataset as a Parquet file
# final_df.to_parquet("./Resources/philly_reviews_without_mood.parquet", index=False)

# Show a preview
print(final_df.head())
print("Saved cleaned dataset to 'https://github.com/tlockhart/project-3/releases/download/v2.0/philly_reviews_without_mood.parquet', length:", len(final_df))

Renamed columns:
Reviews Columns: Index(['review_id', 'user_id', 'business_id', 'review_stars', 'review_useful',
       'funny', 'cool', 'text', 'date'],
      dtype='object')
Users Columns: Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')
Businesses Columns: Index(['business_id', 'business_name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

Reviews DataFrame (first 5 rows):
                review_id                 user_id             business_id  \
0  RC8D-iY19aZFHXUHaS8xLg  11xwVSDv1ytjcyvUiyBWaw  -OI